In [2]:
import sys
sys.path.append('../')
import util.data_processing_tool as dpt

import numpy as np
import properscoring as ps
from datetime import timedelta, date, datetime
import os

ens=np.random.rand(316, 376,20)
obs=np.random.rand(316,376)
####################
# ens=np.random.rand(11,317*376)
# obs=np.random.rand(317*376)
# print(ens,obs)
# ps.crps_ensemble(obs,ens).shape

E:\Users\Weifa\Anaconda3\envs\py37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
E:\Users\Weifa\Anaconda3\envs\py37\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
E:\Users\Weifa\Anaconda3\envs\py37\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
E:\Users\Weifa\Anaconda3\envs\py37\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
import properscoring as ps
import numpy as np
ens=np.random.rand(316, 376,200)
obs=np.random.rand(316,376)
ps.crps_ensemble(obs,ens).shape

MemoryError: Unable to allocate 35.4 GiB for an array with shape (316, 376, 200, 200) and data type float64

In [6]:
!pip install imageio


ERROR: torchvision 0.8.1 requires torch==1.7.0, which is not installed.
ERROR: imgaug 0.2.9 requires matplotlib, which is not installed.
ERROR: imgaug 0.2.9 requires scikit-image>=0.11.0, which is not installed.


In [6]:
from statsmodels.distributions.empirical_distribution import ECDF
def crps(ensin,obs):
    '''
    @param ensin A vector of prediction
    @param obs  A vector of observations
    
'''

#     assert not np.isnan(ensin).any() and not np.isnan(obs).any(), "data contains nan"
         
    Fn = ECDF(ensin)
    xn=np.sort(np.unique(ensin))
    m=len(xn)
    dn=np.diff(xn)
    eq1=0
    eq2=0
    if(obs>xn[0] and obs<xn[m-1]): #obs在范围内
        k=np.max(np.where(xn<=obs))#小于obs的最大值下标
        x0 = xn[k] #小于obs的最大值
        if k>0:
            eq1=np.sum(Fn(xn[0:k+1])**2*np.append(dn[0:k], obs - xn[k]))#小于obs的所有值 的 百分比数 的平方
        else:
            eq1 =np.sum(Fn(xn[0])**2*(obs - xn[0]))
        if k<m-2:

            eq2=np.sum((1-Fn(xn[k:m-1]))**2*np.append(xn[k+1] - obs, dn[(k+1):(m-1)]))
        else:
            eq2 =np.sum((1-Fn(xn[m-2]))**2*(xn[m-1] - obs))

    if obs <= xn[0]: # 观测值在之外
        eq2 =np.sum(np.append(1, 1-Fn(xn[0:(m-1)]))**2*np.append(xn[0]-obs, dn))
    if obs >= xn[m-1]:
        eq1= np.sum(Fn(xn)**2*np.append(dn, obs - xn[m-1]))
            
    return eq1+eq2 




def vectcrps_v(fct_ens,obs):
    '''
    #' @param fct_ens A 2D prediction
    #' @param obs  A vector of observations
    #' @return a crps vector'''
    score =0

    
    fct_ens=fct_ens
    assert not np.isnan(fct_ens).any() and not np.isnan(obs).any(),"data contains nan"
    for i in range(obs.shape[0]):
#         print(fct_ens[:,i],obs[i])
        score+=crps(fct_ens[:,i],obs[i])
  
    return score


def vectcrps_m(fct_ens,obs):
    '''
    #' @param fct_ens A 2D prediction 11*1*1
    #' @param obs  A vector of observations
    #' @return a crps vector'''
    score =0
#     assert np.isnan(fct_ens).any() and np.isnan(obs).any(),"data contains nan"
    score_map=np.zeros((obs.shape[0],obs.shape[1]))
    for i in range(obs.shape[0]):
        for j in range(obs.shape[1]):
            score_map[i,j]=crps(fct_ens[:,i,j],obs[i,j])
#             score+=crps(fct_ens[:,i,j],obs[i,j])
    return score_map

In [6]:
#计算climatology for validation 

# from scipy.stats import norm
# ps.crps_ensemble(obs,ens).shape



###################paramter you need to set###########################
dates_needs=dpt.date_range(date(1997, 1, 1),date(1998, 7, 29))
time_windows=6          #range of the target time as climatology ensemble  rangeshoule be 1-6
file_BARRA_dir="../../Data/barra_aus/"
######################################################################


def rmse(ens,hr):
    '''
    ens:(ensemble,H,W)
    hr: (H,W)
    '''
    return np.sqrt((ens-hr).sum(axis=(0)))

def mae(ens,hr):
    '''
    ens:(ensemble,H,W)
    hr: (H,W)
    '''
    return np.abs((ens-hr)).sum(axis=0)




date_map=np.array(dates_needs)
# np.where(date_map==date(2012, 1, 1))
crps_ref=[]
mae_ref=[]
rmse_ref=[]
import tqdm
for target_date in tqdm.tqdm(dates_needs):
    hr=dpt.read_barra_data_fc(file_BARRA_dir,target_date)
    ensamble=[]
    for y in range(1990,2013):

        if target_date.year==y:
            continue
        
        for w in range(1,time_windows): #for what time of windows

            filename=file_BARRA_dir+str(y)+(target_date-timedelta(w)).strftime("%m%d")+".nc"
            if os.path.exists(filename):
                t=date(y,(target_date-timedelta(w)).month,(target_date-timedelta(w)).day)
                sr=dpt.read_barra_data_fc(file_BARRA_dir,t)
                ensamble.append(sr)

            filename=file_BARRA_dir+str(y)+(target_date+timedelta(w)).strftime("%m%d")+".nc"
            if os.path.exists(filename):

                t=date(y,(target_date+timedelta(w)).month,(target_date+timedelta(w)).day)


                sr=dpt.read_barra_data_fc(file_BARRA_dir,t)
                ensamble.append(sr)

        filename=file_BARRA_dir+str(y)+target_date.strftime("%m%d")+".nc"
        if os.path.exists(filename):
            t=date(y,target_date.month,target_date.day)
            sr=dpt.read_barra_data_fc(file_BARRA_dir,t)
            ensamble.append(sr)
    if ensamble:
        ensamble=np.array(ensamble)
        print(ensamble.shape)
#         a=ps.crps_ensemble(hr,ensamble.transpose(1,2,0))
#         a=vectcrps_m(ensamble,hr)
        rmse_score=rmse(ensamble,hr)
        mae_score=mae(ensamble,hr)
        
        
        mae_ref.append(mae_score)
        rmse_ref.append(rmse_score)
#         crps_ref.append(a)

if not os.path.exists('../save/mae/climatology/'):
    dpt.mkdir('../save/mae/climatology/')

if not os.path.exists('../save/rmse/climatology/'):
    dpt.mkdir('../save/rmse/climatology/')
    
if not os.path.exists('../save/crps/climatology/'):
    dpt.mkdir('../save/crps/climatology/')

# np.save("../save/crps/climatology/climatology_"+str(1997)+"_all_lead_time_windows_11",np.array(crps_ref))
np.save("../save/mae/climatology/climatology_"+str(1997)+"_all_lead_time_windows_11",np.array(mae_ref))
np.save("../save/rmse/climatology/climatology_"+str(1997)+"_all_lead_time_windows_11",np.array(rmse_ref))

# print(" : "+str(np.array(crps_ref).mean()),",")

  0%|          | 0/575 [00:00<?, ?it/s]

(241, 316, 376)


  0%|          | 1/575 [00:22<3:36:48, 22.66s/it]

(241, 316, 376)


  0%|          | 2/575 [00:45<3:36:20, 22.65s/it]

...

100%|█████████▉| 573/575 [3:39:11<00:45, 22.77s/it]

(242, 316, 376)


100%|█████████▉| 574/575 [3:39:34<00:22, 22.73s/it]

(242, 316, 376)


100%|██████████| 575/575 [3:39:56<00:00, 22.70s/it]


In [4]:
#计算climatology for validation 

# from scipy.stats import norm
# ps.crps_ensemble(obs,ens).shape



###################paramter you need to set###########################
dates_needs=dpt.date_range(date(2010, 1, 1),date(2011, 7, 29))
time_windows=6          #range of the target time as climatology ensemble  rangeshoule be 1-6
file_BARRA_dir="../../Data/barra_aus/"
######################################################################






date_map=np.array(dates_needs)
# np.where(date_map==date(2012, 1, 1))
crps_ref=[]
import tqdm
for target_date in tqdm.tqdm(dates_needs):
    hr=dpt.read_barra_data_fc(file_BARRA_dir,target_date)
    ensamble=[]
    for y in range(1990,2013):

        if target_date.year==y:
            continue
        
        for w in range(1,time_windows): #for what time of windows

            filename=file_BARRA_dir+str(y)+(target_date-timedelta(w)).strftime("%m%d")+".nc"
            if os.path.exists(filename):
                t=date(y,(target_date-timedelta(w)).month,(target_date-timedelta(w)).day)
                sr=dpt.read_barra_data_fc(file_BARRA_dir,t)
                ensamble.append(sr)

            filename=file_BARRA_dir+str(y)+(target_date+timedelta(w)).strftime("%m%d")+".nc"
            if os.path.exists(filename):

                t=date(y,(target_date+timedelta(w)).month,(target_date+timedelta(w)).day)


                sr=dpt.read_barra_data_fc(file_BARRA_dir,t)
                ensamble.append(sr)

        filename=file_BARRA_dir+str(y)+target_date.strftime("%m%d")+".nc"
        if os.path.exists(filename):
            t=date(y,target_date.month,target_date.day)
            sr=dpt.read_barra_data_fc(file_BARRA_dir,t)
            ensamble.append(sr)
    if ensamble:
        ensamble=np.array(ensamble)
        print(hr.shape,ensamble.shape)
        a=ps.crps_ensemble(hr,ensamble.transpose(1,2,0))
        a_my=vectcrps_m(ensamble,hr)
        crps_ref.append(a)
        print(a_my.mean(),a.mean())
    break

# if not os.path.exists('../save/crps/climatology/'):
#     dpt.mkdir('../save/crps/climatology/')
# np.save("../save/crps/climatology/climatology_"+str(2010)+"_all_lead_time_windows_11",np.array(crps_ref))
# print(" : "+str(np.array(crps_ref).mean()),",")

  0%|          | 0/575 [00:00<?, ?it/s]

(316, 376) (241, 316, 376)


NameError: name 'vectcrps_m' is not defined

In [3]:
for w in range(1,2):
    print(w)

1


In [1]:
#计算climatology for validation for nci 

# from scipy.stats import norm
# ps.crps_ensemble(obs,ens).shape
import sys
sys.path.append('../')
import util.data_processing_tool as dpt

import numpy as np
import properscoring as ps
from datetime import timedelta, date, datetime
import os


def rmse(ens,hr):
    '''
    ens:(ensemble,H,W)
    hr: (H,W)
    '''
    return np.sqrt( ((ens-hr)**2)  .sum(axis=(0))/ ens.shape[0] )

def mae(ens,hr):
    '''
    ens:(ensemble,H,W)
    hr: (H,W)
    '''
    return np.abs((ens-hr)).sum(axis=0)/ ens.shape[0]



def main(year,time_windows):
    dates_needs=dpt.date_range(date(year, 1, 1),date(year+1, 7, 29))
    file_BARRA_dir="../../Data/barra_aus/"
    date_map=np.array(dates_needs)
    # np.where(date_map==date(2012, 1, 1))
    crps_ref=[]
    mae_ref=[]
    rmse_ref=[]
    import tqdm
    for target_date in tqdm.tqdm(dates_needs):
        hr=dpt.read_barra_data_fc(file_BARRA_dir,target_date)
        ensamble=[]
#         for y in range(1990,target_date.year):1997.1.1: 1998,7,29
        for y in range(1990,2013):
                
            try:
                if date(y,target_date.month,target_date.day)>=date(year, 1, 1) and date(y,target_date.month,target_date.day)<=date(year+1, 7, 29):
                    continue
            except :
                continue

            for w in range(1,time_windows): #for what time of windows

                filename=file_BARRA_dir+str(y)+(target_date-timedelta(w)).strftime("%m%d")+".nc"
                if os.path.exists(filename):
                    t=date(y,(target_date-timedelta(w)).month,(target_date-timedelta(w)).day)
                    sr=dpt.read_barra_data_fc(file_BARRA_dir,t)
                    ensamble.append(sr)

                filename=file_BARRA_dir+str(y)+(target_date+timedelta(w)).strftime("%m%d")+".nc"
                if os.path.exists(filename):

                    t=date(y,(target_date+timedelta(w)).month,(target_date+timedelta(w)).day)


                    sr=dpt.read_barra_data_fc(file_BARRA_dir,t)
                    ensamble.append(sr)

            filename=file_BARRA_dir+str(y)+target_date.strftime("%m%d")+".nc"
            if os.path.exists(filename):
                t=date(y,target_date.month,target_date.day)
                sr=dpt.read_barra_data_fc(file_BARRA_dir,t)
                ensamble.append(sr)
        if ensamble:
            ensamble=np.array(ensamble)

            a=ps.crps_ensemble(hr,ensamble.transpose(1,2,0))
    #         a=vectcrps_m(ensamble,hr)
            rmse_score=rmse(ensamble,hr)
            mae_score=mae(ensamble,hr)


            mae_ref.append(mae_score)
            rmse_ref.append(rmse_score)
            crps_ref.append(a)

    if not os.path.exists('../save/mae/climatology/'):
        dpt.mkdir('../save/mae/climatology/')

    if not os.path.exists('../save/rmse/climatology/'):
        dpt.mkdir('../save/rmse/climatology/')

    if not os.path.exists('../save/crps/climatology/'):
        dpt.mkdir('../save/crps/climatology/')

    np.save("../save/crps/climatology/climatology_"+str(year)+"_varify_lead_time_windows_"+str((time_windows-1)*2+1),np.array(crps_ref))
    np.save("../save/mae/climatology/climatology_"+str(year)+"_varify_lead_time_windows_"+str((time_windows-1)*2+1),np.array(mae_ref))
    np.save("../save/rmse/climatology/climatology_"+str(year)+"_varify_lead_time_windows_"+str((time_windows-1)*2+1),np.array(rmse_ref))

    print(year,time_windows)

E:\Users\Weifa\Anaconda3\envs\py37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
E:\Users\Weifa\Anaconda3\envs\py37\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
E:\Users\Weifa\Anaconda3\envs\py37\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
E:\Users\Weifa\Anaconda3\envs\py37\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
if __name__=='__main__':
    year_list=[1997,2010,2012]
    timewind=[1,11]
    for i in timewind:
        for j in year_list:
            main(j,i)

100%|████████████████████████████████████████████████████████████████████████████████| 575/575 [03:36<00:00,  4.99it/s]


1997 1


100%|████████████████████████████████████████████████████████████████████████████████| 575/575 [01:41<00:00,  5.78it/s]


2010 1


100%|████████████████████████████████████████████████████████████████████████████████| 576/576 [01:50<00:00,  4.47it/s]


2012 1



\begin{equation}
Triplet\_loss=
\sum_{i}^{N}\left[\left\|\left(x_{i}^{Anchor}\right)-f\left(x_{i}^{positive}\right)\right\|_{2}^{2}-\left\|f\left(x_{i}^{Anchor}\right)-f\left(x_{i}^{Negative}\right)\right\|_{2}^{2}+\alpha\right]_{+}
\end{equation}

In [2]:
year=1997
a=date(year, 1, 1)
b=date(year+1, 7, 29)

a<b

True

In [ ]:
np.load("../save/crps/climatology/climatology_1997_all_lead_time_windows_11.npy")
np.load("../../vdsr/save/crps/c")